In [258]:
# read JSON file
import json
import pandas as pd
import simplefix

# read file
top_jsondf = pd.read_json('top.json')

#replace NaN with blanks
top_jsondf.fillna('',  inplace=True)

#transpose dataframe
top_jsondf = top_jsondf.T

In [318]:
# download https://github.com/Morgan-Stanley/pyfixmsg

# from https://pyfixmsg.readthedocs.io/en/latest/example.html

from pyfixmsg import RepeatingGroup
from pyfixmsg.fixmessage import FixMessage, FixFragment
from pyfixmsg.reference import FixSpec, FixTag
from pyfixmsg.codecs.stringfix import Codec

import ast
import numpy as np

# set specifications for FIX message
# download specification file from https://github.com/quickfix/quickfix/blob/master/spec/FIX42.xml
spec = FixSpec('fix42.xml')
codec = Codec(spec=spec,fragment_class=FixFragment)

# read top.txt
top_txt = [line.rstrip('\n') for line in open('top.txt')]

# FIX42 has 400 fields in total. So for...range generates 446 numbers and list..str converts it to string
all_fields = list(str(i) for i in range(1,447))
missing = ['24','51','85','86','101','125','220','221','222','224','225','226','227','228','229','230','232','233','234','235','236','237','238','239','240','241','242','243','244','245','246','247','248','249','250','251','252','253','254','255','256','257','258','259','260','261']
all_fields = sorted(set(all_fields) - set(missing))

top_fixdf = pd.DataFrame(columns=all_fields)

for toplist in top_txt:
    # get FIX message from the top txt file
    fixmsg = FixMessage().load_fix(bytes(toplist.replace('"',""),'utf-8'),separator=",")
    
    #parse fix message
    fix_str = ast.literal_eval(str(fixmsg))
    fix_str_split = fix_str.decode('utf-8').split(';')

    # create dataframe from the FIX message
    fix_list = []

    for i in fix_str_split[:-1]:
        fix_list.append(i.split("="))

    templist_fixdf=pd.DataFrame(fix_list,columns=['Tag','Values'])
    top_fixdf_temp = pd.DataFrame(temp_fixdf['Values'].tolist(), index=temp_fixdf['Tag'].tolist()).T.reindex(columns=all_fields, fill_value='')
    
    top_fixdf = pd.concat([top_fixdf, top_fixdf_temp])

# # drop columns with all empty columns
# top_fixdf = top_fixdf.dropna(how='all', axis='columns')

In [319]:
top_fixdf[['9','35','49','56','34','10','52','109','40']]

,9,35,49,56,34,10,52,109,40
0,46,6,ABC,CAB,14,205,20100204-09:18:42,,
0,46,6,ABC,CAB,14,205,20100204-09:18:42,,
0,46,6,ABC,CAB,14,205,20100204-09:18:42,,
0,46,6,ABC,CAB,14,205,20100204-09:18:42,,


In [306]:
# to get names of message types
top_fixdf['35'].apply(lambda x: spec.msg_types[x].name if x != '' else '')

0    IOI
0    IOI
0    IOI
0    IOI
Name: 35, dtype: object

In [308]:
# to get Field names
nameField = lambda x: spec.tags.by_tag(int(x)).name
top_fixdf_fieldnames = list(map(nameField, top_fixdf.columns))
top_fixdf_fieldnames

['Account',
 'CheckSum',
 'ExDestination',
 'CxlRejReason',
 'OrdRejReason',
 'IOIQualifier',
 'WaveNo',
 'Issuer',
 'SecurityDesc',
 'HeartBtInt',
 'ClientID',
 'ClOrdID',
 'MinQty',
 'MaxFloor',
 'TestReqID',
 'ReportToExch',
 'LocateReqd',
 'OnBehalfOfCompID',
 'OnBehalfOfSubID',
 'QuoteID',
 'NetMoney',
 'SettlCurrAmt',
 'Commission',
 'SettlCurrency',
 'ForexReq',
 'OrigSendingTime',
 'GapFillFlag',
 'NoExecs',
 'ExpireTime',
 'DKReason',
 'DeliverToCompID',
 'DeliverToSubID',
 'CommType',
 'IOINaturalFlag',
 'QuoteReqID',
 'BidPx',
 'OfferPx',
 'BidSize',
 'OfferSize',
 'NoMiscFees',
 'MiscFeeAmt',
 'MiscFeeCurr',
 'MiscFeeType',
 'CumQty',
 'PrevClosePx',
 'ResetSeqNumFlag',
 'SenderLocationID',
 'TargetLocationID',
 'OnBehalfOfLocationID',
 'DeliverToLocationID',
 'NoRelatedSym',
 'Subject',
 'Headline',
 'URLLink',
 'Currency',
 'ExecType',
 'LeavesQty',
 'CashOrderQty',
 'AllocAvgPx',
 'AllocNetMoney',
 'SettlCurrFxRate',
 'SettlCurrFxRateCalc',
 'NumDaysInterest',
 'AccruedI